In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pylab as plot
import numpy as np
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from matplotlib import style
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier,XGBRFRegressor,XGBRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score as cvs


In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
house_price = pd.read_csv('/kaggle/input/boston-house-prices/housing.csv', header=None, delimiter=r"\s+", names=column_names)
house_price.head()

In [ ]:
house_price.shape

In [ ]:
house_price.describe()

### Interpreting Data Description

1. There is no missing values
2. Variable 'ZN' is 0 for 25th and 50th percentile that will result in skweed data
3. Also for variable 'CHAS' it's 0 for 25th, 50th and 75th percentile that will also show us that data is highly skweed.

## Checking  outliers in the data

In [ ]:
#Plotting boxplots to see if there are any outliers in our data (considering data betwen 25th and 75th percentile as non outlier)
fig, ax = plt.subplots(ncols=7, nrows=2, figsize=(15, 5))
ax = ax.flatten()
index = 0
for i in house_price.columns:
    sns.boxplot(y=i, data=house_price, ax=ax[index])
    index +=1
plt.tight_layout(pad=0.4)
plt.show()

Columns CRIM, ZN, RM, DIS, PTRATIO, B, LSTAT and MEDV have outliers.

### Data Exploration

In [ ]:
sns.pairplot(house_price)

1. we can infer from the histogram that the ‘MEDV’ variable seems to be normally distributed but contain several outliers.
2. We can spot a linear relationship between ‘RM’ and House prices ‘MEDV’

In [ ]:
#Establishing correlation in data

from matplotlib import style
#creating a correlation matrix

style.use("classic")
sns.heatmap(house_price.corr(),annot=True,cmap='inferno')

To fit a regression model, the features of interest are the ones with a high correlation with the target variable ‘MEDV’

##  Preprocessing

In [ ]:
x=house_price.drop("MEDV",axis=1)
y=house_price["MEDV"]
x.shape

### feature scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_std, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

## Assessing feature importance with random forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot
forest = RandomForestRegressor(n_estimators=500, random_state = 0)
forest.fit(X_train,y_train)
importances = forest.feature_importances_
feat_labels = house_price.columns[1:]
indices = np.argsort(importances)[::-1]

# plot feature importance
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))


In [ ]:
#removing variables 'ZN' and 'CHAS' form data
house_price = house_price.drop(['CHAS', 'CHAS', 'TAX', 'INDUS', 'NOX'], axis=1)

In [ ]:
house_price.head()

## Train a model

## LinearRegression

In [ ]:
#fitting our model to train and test
lm = LinearRegression()
model = lm.fit(X_train,y_train)

In [ ]:
pred_y = lm.predict(X_test)
Y_compare_linear= pd.DataFrame({"Actual": y_test, "Predict": pred_y})
Y_compare_linear.head()

In [ ]:
print(r2_score(y_test,pred_y))
print(mean_absolute_error(y_test,pred_y))
print(mean_squared_error(y_test,pred_y))

## XGBRegressor

In [ ]:
xgb=XGBRegressor(n_estimators=1000)
xgb.fit(X_train,y_train)
kfold=KFold(n_splits=5)
res=cross_val_score(model,X_train,y_train,cv=kfold)

In [ ]:
yp=xgb.predict(X_test)
Y_compare_XGBReg= pd.DataFrame({"Actual": y_test, "Predict": yp})
Y_compare_XGBReg.head()

In [ ]:
print(r2_score(y_test,yp))
print(mean_absolute_error(y_test,yp))
print(mean_squared_error(y_test,yp))
print(r2_score(y_test,yp))
print(mean_absolute_error(y_test,yp))
print(mean_squared_error(y_test,yp))

## K-Nearest Neighbour

In [ ]:
knn = KNeighborsRegressor(n_neighbors=13)
knn.fit(X_train,y_train)
Y_pred = knn.predict(X_test)
Y_compare_knn = pd.DataFrame({'Actual': y_test, 'Predicted': Y_pred})
Y_compare_knn.head()

In [ ]:
print(r2_score(y_test,Y_pred))
print(mean_absolute_error(y_test,Y_pred))
print(mean_squared_error(y_test,Y_pred))

## Random Forrest Regression

In [ ]:
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train,y_train)
Y_pred = rf.predict(X_test)
Y_compare_randomforrest = pd.DataFrame({'Actual': y_test, 'Predicted': Y_pred})
Y_compare_randomforrest.head() #displaying the comparision btween actual and predicted values of MEDV

In [ ]:
print(r2_score(y_test,Y_pred))
print(mean_absolute_error(y_test,Y_pred))
print(mean_squared_error(y_test,Y_pred))

## Evaluation 

Plotting compariasion of actual and predicted values of MEDV that we got using different machine learning models

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=1, figsize=(25, 4))
ax = ax.flatten()
Y_compare_linear.head(10).plot(kind='bar', title='Linear Regression', grid=True, ax=ax[0])
Y_compare_XGBReg.head(10).plot(kind='bar', title='XGBRegressor', grid=True, ax=ax[1])
Y_compare_randomforrest.head(10).plot(kind='bar', title='Random Forrest Regression', grid=True, ax=ax[2])
Y_compare_knn.head(10).plot(kind='bar', title='KNN Regression', grid=True, ax=ax[3])
plt.show()

Scores (R squared) of different machine learning models using K-fold cross validation:

In [ ]:
print('According to R squared scorring method we got below scores for out machine learning models:')
modelNames = ['Linear Regression', 'XGBRegressor', 'Random Forrest', 'K-Nearest Neighbour']
modelRegressors = [lm, xgb, rf, knn]
models = pd.DataFrame({'modelNames' : modelNames, 'modelRegressors' : modelRegressors})
counter=0
score=[]
for i in models['modelRegressors']:
    accuracy = cvs(i, X_train, y_train, scoring='r2', cv=5)
    print('Accuracy of %s Regression model is %.2f' %(models.iloc[counter,0],accuracy.mean()))
    score.append(accuracy.mean())
    counter+=1

In [ ]:
pd.DataFrame({'Model Name' : modelNames,'Score' : score}).sort_values(by='Score', ascending=True).plot(x=0, y=1, kind='bar', figsize=(15,5), title='Comparison of R2 scores of differnt models', )
plt.show()

we can summarise that Random Forrest (r2 =  0.87) machine learning model gives the best score